# Collective communications

In [4]:
import numpy as np
import ipyparallel as ipp
c = ipp.Client(profile='mpi')
print(c.ids)
view = c[:]
view.activate()

[0, 1, 2, 3]


## Bcast/bcast - przykład

In [25]:
%%px --block 

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    a = 12.
else:
    a = 0.0

if rank>0:
    print(a)
    
a = comm.bcast(a, root=0)

if rank>0:
    print(a)

[stdout:0] 
0.0
12.0
[stdout:1] 
0.0
12.0
[stdout:2] 
0.0
12.0


In [27]:
%%px --block 

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD 
rank = comm.rank

if rank == 0:
    a = np.array([12.],dtype=np.float32)    
else:
    a = np.zeros(1,dtype=np.float32)    

if rank>0:
    print(a)
    
comm.Bcast(a, root=0)

if rank>0:
    print(a)

[stdout:0] 
[ 0.]
[ 12.]
[stdout:1] 
[ 0.]
[ 12.]
[stdout:2] 
[ 0.]
[ 12.]


## Gather/scatter - przykład

In [5]:
%%px --block 

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.rank

N_local = 5
N = N_local*comm.size

if rank == 0:
    A = np.arange(N, dtype=np.float64)
else:
    A = None

A_local = np.empty(N_local)

comm.Scatter( A, A_local )
A_local = A_local+100.0
comm.Gather( A_local, A)

if rank==0:
    print(A)
    


[stdout:3] 
[ 100.  101.  102.  103.  104.  105.  106.  107.  108.  109.  110.  111.
  112.  113.  114.  115.  116.  117.  118.  119.]


In [7]:
view['A_local']

[array([ 110.,  111.,  112.,  113.,  114.]),
 array([ 105.,  106.,  107.,  108.,  109.]),
 array([ 115.,  116.,  117.,  118.,  119.]),
 array([ 100.,  101.,  102.,  103.,  104.])]

## Reduce - przykład

In [17]:
%%px --block 

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.rank

A = rank*np.ones(1,dtype=np.float)

suma = np.zeros(1)
comm.Reduce(A,suma, op = MPI.SUM)
if rank==0:
    print(suma)


[stdout:3] [ 6.]


In [18]:
0+1+2+3

6

In [19]:
view['suma']

[array([ 0.]), array([ 0.]), array([ 0.]), array([ 6.])]

In [20]:
%%px --block 

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD

rank = comm.Get_rank()
rankF = numpy.array(rank,dtype=np.float)
total = numpy.zeros(1)
comm.Reduce(rankF,total, op=MPI.SUM)
if rank==0:
    print(total)

[stdout:3] [ 6.]
